# Preprocessing start

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

import PIL
from PIL import Image

import tensorflow as tf
import keras
from keras.preprocessing import image

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils
from keras.layers import BatchNormalization
from keras import regularizers
from keras.layers import ZeroPadding2D
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2

In [114]:
# To stop potential randomness
seed = 128
rng = np.random.RandomState(seed)

In [115]:
root_dir = os.path.abspath('C:\\Users\\LYC\\Desktop\\SFU\\Fall 2020 Courses\\STAT 440\\Module 3') 
data_dir = os.path.join(root_dir, 'tr') 

In [43]:
train = os.path.join(data_dir, 'img')
# test = pd.read_csv(os.path.join(data_dir, 'Test.csv'))


In [116]:
def loadImages(path):
    '''Put files into lists and return them as one list with all images 
     in the folder'''
    image_files = sorted([os.path.join(path, 'img', file)
                          for file in os.listdir(path + "/img")
                          if file.endswith('.jpg')])
    return image_files

img = loadImages(data_dir)

def load_test_Images(path):
    '''Put files into lists and return them as one list with all images 
     in the folder'''
    image_files = sorted([os.path.join(path, 'te', file)
                          for file in os.listdir(path + "/te")
                          if file.endswith('.jpg')])
    return image_files


In [117]:
def loadLbl(path):
    '''Put files into lists and return them as one list with all images 
     in the folder'''
    lbl_files = sorted([os.path.join(path, 'lbl', file)
                          for file in os.listdir(path + "/lbl")
                          if file.endswith('.txt')])
    return lbl_files

lbl = loadLbl(data_dir)

In [7]:
labels = []

for i in range(len(lbl)):
    img2 = img[i][-11:]
    label = open(lbl[i], "r").read()
    label = label.replace("\n", "")
    labels.append((img2, label))

In [8]:
labels = pd.DataFrame(labels, columns=['name', 'label'])
labels.to_csv('labels.csv', index=False)

In [118]:
train_data = pd.read_csv(os.path.join(root_dir, 'labels.csv'))

In [315]:
temp = []
for img_name in img:
    pic = cv2.imread(img_name)
    height = 56
    width = 56
    dim = (width, height)
    res = cv2.resize(pic, dim, interpolation=cv2.INTER_LINEAR)
    blur = cv2.GaussianBlur(res, (5, 5), 0)
    a = image.img_to_array(blur)
    a = a.astype('float32')
    a /= 255.0
    temp.append(a)
    
train_x = np.stack(temp)

# train_x = train_x.reshape(-1, 784).astype('float32')

# temp = []
# for img_name in test.filename:
#     image_path = os.path.join(data_dir, 'Train', 'Images', 'test', img_name)
#     img = imread(image_path, flatten=True)
#     img = img.astype('float32')
#     temp.append(img)
    
# test_x = np.stack(temp)

# test_x /= 255.0
# test_x = test_x.reshape(-1, 784).astype('float32')

In [316]:
train_og_y = tf.keras.utils.to_categorical(train_data.label.values, num_classes=7, dtype="int")

# Preprocessing Ends

In [317]:
split_size = int(train_x.shape[0]*0.7)

In [318]:
train_x, val_x = train_x[:split_size], train_x[split_size:]

In [319]:
train_y, val_y = train_og_y[:split_size], train_og_y[split_size:]

In [320]:
train_x.shape

(884, 56, 56, 3)

In [321]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils
from keras.layers import BatchNormalization
from keras import regularizers
from keras.layers import ZeroPadding2D
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2

In [356]:
# model architecture

# model = Sequential()
# model.add(ZeroPadding2D(padding=(1, 1)))
# model.add(Conv2D(32, kernel_size=(4, 4),activation='relu',strides = 2, input_shape=(56,56,3),padding='valid'))
# model.add(Conv2D(64, kernel_size=(3, 3),activation='relu',strides = 1))
# model.add(MaxPool2D(pool_size=(2, 2)))
# model.add(BatchNormalization())
# model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPool2D(pool_size=(2, 2)))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(7, activation='softmax'))

model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3),activation='relu',input_shape=(56,56,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides = 2))
model.add(Conv2D(64, (3,3), activation='relu', strides = 2))
model.add(Conv2D(32, (2,2), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.1))
model.add(Conv2D(32, (2,2), activation='relu'))
model.add(MaxPool2D(pool_size=(1, 1)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(7, activation='softmax'))

In [357]:
model.compile(loss='categorical_crossentropy', optimizer=tf.optimizers.Adamax(lr=0.0009), metrics=['accuracy'])

In [358]:
model.fit(train_x, train_y, epochs=60, validation_data=(val_x, val_y))


Epoch 1/60
28/28 [==============================] - 21s 744ms/step - loss: 1.8211 - accuracy: 0.2262 - val_loss: 1.7693 - val_accuracy: 0.2401
Epoch 2/60
28/28 [==============================] - 23s 836ms/step - loss: 1.7691 - accuracy: 0.2285 - val_loss: 1.7627 - val_accuracy: 0.2401
Epoch 3/60
28/28 [==============================] - 21s 747ms/step - loss: 1.7694 - accuracy: 0.2081 - val_loss: 1.7780 - val_accuracy: 0.2375
Epoch 4/60
28/28 [==============================] - 21s 757ms/step - loss: 1.7559 - accuracy: 0.2432 - val_loss: 1.7432 - val_accuracy: 0.2507
Epoch 5/60
28/28 [==============================] - 21s 754ms/step - loss: 1.7575 - accuracy: 0.2070 - val_loss: 1.7287 - val_accuracy: 0.2375
Epoch 6/60
28/28 [==============================] - 21s 744ms/step - loss: 1.7397 - accuracy: 0.2217 - val_loss: 1.7037 - val_accuracy: 0.2823
Epoch 7/60
28/28 [==============================] - 21s 752ms/step - loss: 1.6706 - accuracy: 0.2851 - val_loss: 1.6324 - val_accuracy: 0.2929

In [94]:
# prepare test data
#load test set
test = load_test_Images(root_dir)
#read and store test image
test_image = []
for i in test:
    pic = image.load_img(i, target_size=(227,227,1), grayscale=False)
    a = image.img_to_array(pic)
    a = a.astype('float32')
    a /= 255.0
    test_image.append(a)
    
test = np.stack(test_image)

# making predictions
prediction = model.predict_classes(test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([5, 4, 2, ..., 5, 5, 5], dtype=int64)

In [95]:
from pandas import *
idx = pd.Series(Int64Index(range(1264,2528)))
prediction = pd.Series(prediction)

data = concat([idx,prediction],axis = 1)
data.columns = ['Id',"Prediction"]


In [96]:
#creating submission file
data.to_csv('kaggle_submission.csv', header=True, index=False)